# Rank-BM25 Library

In [1]:
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Reading and processing the CISI.ALL file, which contains the documents to be searched/retrieved.

In [3]:
with open('CISI.ALL', 'r') as file:
    content = file.read()

In [4]:
documents = content.split('.I ')

For each document, the title, author, and text are concatenated to assist in information retrieval.

In [5]:
docs = []
for i, document in enumerate(documents):
  title = document[document.find('\n.T') + 3:document.find('\n.A')].strip()
  author = document[document.find('\n.A') + 3: document.find('\n.W')].strip()
  text = document[document.find('\n.W') + 3: document.find('\n.X')].strip()
  doc = title + ' ' + author + ' ' + text
  docs.append(doc)

Reading and processing the CISI.QRY file, which contains the queries.

In [6]:
with open('CISI.QRY', 'r') as file:
    content = file.read()

In [7]:
queries = content.split('.I ')

In [8]:
query_docs = []
for query in queries:
  text = query[query.find('.W\n') + 3:].strip()
  query_docs.append(text)

Tonekizing the documents

In [9]:
from rank_bm25 import BM25Okapi, BM25L, BM25Plus, BM25

corpus = docs

tokenized_corpus = [doc.split(" ") for doc in corpus]

Reading and processing of the CISI.REL file, which contains the reference/target relevance pairs that relate the queries to the documents.

In [10]:
from collections import defaultdict

map_query_to_docs = defaultdict(list)
total = 0

with open('CISI.REL', 'r') as file:
    for line in file:
        cols = line.split()
        query_id = cols[0]
        doc_id = cols[1]
        map_query_to_docs[int(query_id)].append(int(doc_id))
        total += 1

Next, the code responsible for executing the algorithm from the document base is shown.

In [14]:
import numpy as np

def calculate_bm25(bm25, k1, b):
  """
  Função repsponsável por testar o algoritmo BM25 sobre a base de documentos.

  Parameters
  -----------
  bm25: instância do modelo (implementação/variante específica do algoritmo 
        BM25), previamente instanciada sobre a base de documentos.
  Returns
  -------
  Tupla contendo MAP (mean average precision) e recall obtidos pelo modelo.
  """
  threshold = 0
  query_precision = defaultdict(list)
  query_recall = defaultdict(list)

  for i in range(1, len(query_docs)):
      query = query_docs[i]
      tokenized_query = query.split(" ")
      doc_scores = bm25.get_scores(tokenized_query)

      #quanto maior este valor, melhor o MAP
      threshold = np.percentile(doc_scores, 95)
      #threshold = np.percentile(doc_scores, 75)
      
      relevant_docs = map_query_to_docs[i]
      if len(relevant_docs) > 0:
        retrieved_docs = [i for i, doc_score in enumerate(doc_scores) 
                            if doc_score >= threshold]
        relevant_retrieved = 0
        for doc in relevant_docs:
          if doc in retrieved_docs:
            relevant_retrieved += 1
        recall = relevant_retrieved/len(relevant_docs)
        query_recall[i].append(recall)

        retrieved_relevant = 0
        for doc in retrieved_docs:
          if doc in relevant_docs:
            retrieved_relevant += 1
        precision = retrieved_relevant/len(retrieved_docs)
        query_precision[i].append(precision)

  map = 0
  recall = 0
  n = 0
  for q in query_precision:
    mean_query_precision = sum(query_precision[q])/len(query_precision[q])
    map += mean_query_precision
    mean_query_recall = sum(query_recall[q])/len(query_recall[q])
    recall += mean_query_recall
    n += 1

  print('Model = ', bm25)
  print('k1 = ', k1)
  print('b = ', b)
  map = map/n
  recall = recall/n
  print('MAP: ', map)
  print('Mean recall: ', recall)
  return map, recall
        

Next, there is a grid search that varies:

1. the specific variant of the BM25 algorithm implementation,
2. the k1 hyperparameter, and
3. the b hyperparameter.

In [15]:
best_map = 0
best_k1 = None
best_b = None
best_model = None

for k1 in [0.5, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]:
  for b in [0.25, 0.5, 0.75, 1.0]:
    for model in [BM25Okapi(tokenized_corpus, k1=k1, b=b), 
                  BM25L(tokenized_corpus, k1=k1, b=b), 
                  BM25Plus(tokenized_corpus, k1=k1, b=b)]:
      map, _ = calculate_bm25(model, k1, b)
      if map > best_map:
        best_map = map
        best_k1 = k1
        best_b = b
        best_model = model

print('Best MAP = ', best_map)
print('Best k1 = ', best_k1)
print('Best b = ', best_b)
print('Best model = ', best_model)

Model =  <rank_bm25.BM25Okapi object at 0x7f1bda539040>
k1 =  0.5
b =  0.25
MAP:  0.10326953748006375
Mean recall:  0.11132806250868306
Model =  <rank_bm25.BM25L object at 0x7f1bf0d438b0>
k1 =  0.5
b =  0.25
MAP:  0.08971291866028704
Mean recall:  0.09786346812120587
Model =  <rank_bm25.BM25Plus object at 0x7f1bf0d35a30>
k1 =  0.5
b =  0.25
MAP:  0.11004784688995213
Mean recall:  0.12000110188969781
Model =  <rank_bm25.BM25Okapi object at 0x7f1bf0d434c0>
k1 =  0.5
b =  0.5
MAP:  0.10287081339712914
Mean recall:  0.11119186650896905
Model =  <rank_bm25.BM25L object at 0x7f1bda539190>
k1 =  0.5
b =  0.5
MAP:  0.09051036682615625
Mean recall:  0.09791162861436933
Model =  <rank_bm25.BM25Plus object at 0x7f1bda539bb0>
k1 =  0.5
b =  0.5
MAP:  0.11084529505582134
Mean recall:  0.1217062650823362
Model =  <rank_bm25.BM25Okapi object at 0x7f1bda5396a0>
k1 =  0.5
b =  0.75
MAP:  0.10446570972886758
Mean recall:  0.11787236099024917
Model =  <rank_bm25.BM25L object at 0x7f1bda539190>
k1 =  0.5


In [16]:
map, recall = calculate_bm25(BM25Plus(tokenized_corpus, k1=best_k1, b=best_b), best_k1, best_b)
print('MAP = ', map)
print('Recall = ', recall)
print('F-1 = ', 2*map*recall/(map + recall))

Model =  <rank_bm25.BM25Plus object at 0x7f1bda5392e0>
k1 =  3.0
b =  0.75
MAP:  0.1164274322169059
Mean recall:  0.1211214977826559
MAP =  0.1164274322169059
Recall =  0.1211214977826559
F-1 =  0.11872808665672628


OBS.: With threshold = 75th percentile, the following values were obtained:

* Model =  <rank_bm25.BM25Plus object at 0x7f1bda504490>
* k1 =  1.5
* b =  0.5
* MAP:  0.06221572449642625
* Mean recall:  0.29705628218503843
* MAP =  0.06221572449642625
* Recall =  0.29705628218503843
* F-1 =  0.10288345024745002